# This stuff is the by the book implementation

In [2]:
import numpy as np
import itertools
import math

In [3]:
np.set_printoptions(linewidth=200)

In [4]:
def choose_ab(a, b):
    return a / b

In [5]:
def choose_both(a, b):
    return choose_ab(a, b) + choose_ab(b, a)

In [6]:
def f_first(vals1, vals2):
    prod = np.complex128(1)
    for a in vals1:
        for b in vals2:
            prod *= choose_both(a, b)

    return prod

In [7]:
def f_second(vals1, vals2):
    n1 = len(vals1)
    nboth = n1 + len(vals2)
    A = np.zeros([nboth, nboth], dtype=np.complex128)
    for j, a in enumerate(vals1):
        for (k, b) in enumerate(vals2):
            A[j,k+n1] = -1 * choose_ab(a, b) / choose_both(a, b)
            A[k+n1,j] = -1 * choose_ab(b, a) / choose_both(a, b)

    for i in range(nboth):
        # check if this is row or col sum
        A[i, i] = -1 * A[i, :].sum()

    return np.linalg.det(A[:-1, :-1])

In [8]:
def combs_with_multiplicity_recip(vals, n):
    if n == 0:
        yield (1, [])
    elif len(vals) == 1:
        yield (math.factorial(n), [vals[0]] * n)
    else:
        for i in range(0, n+1):
            for (rest_recip, rest_vals) in combs_with_multiplicity_recip(vals[1:], n-i):
                yield (rest_recip * math.factorial(i), [vals[0]] * i + rest_vals)

In [9]:
def combs_with_multiplicity(vals, n):
    fact = math.factorial(n)
    for recip, vals in combs_with_multiplicity_recip(vals, n):
        yield fact // recip, vals

In [10]:
def f(vals1, vals2):
    return f_first(vals1, vals2) * f_second(vals1, vals2)

In [11]:
def make_roots(m):
    return np.roots([1] + [0]*(m-1) + [-1])

In [12]:
def m_for(n):
    return 2*((n+3) // 4) +1

In [13]:
def count_cycles(n1, n2):
    # these are deliberately switched
    m1 = m_for(n2)
    m2 = m_for(n1)

    s = np.complex128()

    for multx, x in combs_with_multiplicity(make_roots(m1), n1):
        for multy, y in combs_with_multiplicity(make_roots(m2), n2-1):
            s += multx * multy * f(x, y +  [np.complex128(1)])
    
    return s / (m1 ** (n1)) / (m2 ** (n2-1))

In [20]:
def is_prime(x):
    for i in range(2, int(math.sqrt(x) + 1)):
        if x % i == 0:
            return False
    return True

In [23]:
primes = [x for x in range(2, 1000) if is_prime(x)]

In [36]:
def factorize(n):
    n_init = n
    s = ""
    for p in primes:
        c = 0
        while n % p == 0:
            n //= p
            c += 1
        if c != 0:
           s += f" * {p}^{c}"
    return f"{n_init} =" + s[2:]

In [49]:
def analyse(*args):
    return factorize(round(count_cycles(*args).real))

In [63]:
def best_coeff(n1, n2):
    return (math.factorial(n1-1)**n2) * (math.factorial(n2-1)**n1)

In [68]:
best_coeff(6, 6)

8916100448256000000000000

In [45]:
round(count_cycles(4, 4).real)

6336

In [40]:
# 4, 6


TypeError: unsupported operand type(s) for %: 'numpy.complex128' and 'int'